In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
import nvtabular as nvt
import cudf
import os
from config import project_dir,data_dir_raw
import pandas as pd
import dask_cudf
from nvtabular.ops import *

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
# files= os.listdir(os.path.join(project_dir, data_dir_raw,'amazon'))
# for file in files:
#     csv = cudf.read_csv(os.path.join(project_dir, data_dir_raw,'amazon',file))
#     csv.to_parquet(os.path.join(project_dir, data_dir_raw,'amazon',file.replace('.csv','.parquet')))

train_data_dir = os.path.join(project_dir, data_dir_raw,'amazon')
test_data_dir = os.path.join(project_dir, data_dir_raw,'amazon')
task = 'task1'
PREDS_PER_SESSION = 100

In [2]:
products = cudf.read_parquet(os.path.join(project_dir, data_dir_raw,'amazon','products_train.parquet'))
test_sessions = cudf.read_parquet(os.path.join(project_dir, data_dir_raw,'amazon','sessions_test_task1.parquet'))
train_sessions = cudf.read_parquet(os.path.join(project_dir, data_dir_raw,'amazon','sessions_train.parquet')) 

In [3]:
def str2list(x):
    return eval(x.replace(' ',','))

In [4]:
# explode the prev_items column, so that each row has one item but keep the order of the items in the session by creating an index column
train_sessions.prev_items = train_sessions.prev_items.to_pandas().apply(str2list)
train_sessions['id']= train_sessions.index
train_sessions= train_sessions.explode('prev_items', ignore_index=True).sort_values('id')

In [11]:
nunique = train_sessions.id.nunique() * 0.75
train = train_sessions[train_sessions.id < int(nunique)]
val = train_sessions[train_sessions.id >= int(nunique)]

train.to_parquet(train_data_dir + '/train.parquet')
val.to_parquet(train_data_dir + '/val.parquet')

START MODELING

In [2]:
train = cudf.read_parquet(train_data_dir + '/train.parquet')
val = cudf.read_parquet(train_data_dir + '/val.parquet')
products = cudf.read_parquet(os.path.join(project_dir, data_dir_raw,'amazon','products_train.parquet'))
products.rename(columns={'id':'prev_items'},inplace=True)
train=train.loc[train.id < 1000]
val=val.loc[val.id < 100]

In [3]:
columns= ['prev_items',  'price', 'brand', 'color', 'size', 'model','material', 'author']
train = train.merge(products[columns], how='left',on='prev_items')
val = val.merge(products[columns], how='left',on='prev_items')
del products

In [18]:
columns = train.columns

In [4]:
categorical = ['prev_items', 'next_item', 'locale', 'brand', 'color', 'model', 'material', 'author']
numerical = ['price','size']

In [5]:
categorical_list = [nvt.ColumnSchema(name=col, tags=[Tags.CATEGORICAL]) for col in categorical]
categorical_processed = [categorical] >> Categorify()
# numerical_list = [nvt.ColumnSchema(name=col, tags=[Tags.CONTINUOUS]) for col in numerical]
# numerical_processed = [numerical] >> FillMissing() >> Clip(min_value=0) >> LogOp() >> Normalize() 
# processor = nvt.Workflow(categorical_processed + numerical_processed)
# processor.fit(train, columns=categorical_list + numerical_list)

In [8]:
processor = nvt.Workflow(categorical_processed)
processor.fit(train)

AttributeError: DataFrame object has no attribute schema

In [14]:
numerical = [numerical] >> FillMissing() >> Clip(min_value=0) >> Normalize() >> categorical_list

,prev_items,next_item,locale,id,price,brand,color,size,model,material,author
0,B0967LF6KW,B096TZRWMZ,DE,265,17.99,CASEKOO,clear,<NA>,iPhone 13 Pro,Polykarbonat,<NA>
1,B0967LF6KW,B096TZRWMZ,DE,265,15.98,CASEKOO,Clear,<NA>,iPhone 13 Pro,Polycarbonate,<NA>
2,B09WJ1NBHQ,B096TZRWMZ,DE,265,19.99,CASEKOO,Alpingrün,<NA>,iPhone 13 Pro,Kunststoff,<NA>
3,B09WJ1NBHQ,B096TZRWMZ,DE,265,18.99,CASEKOO,Alpine Green,<NA>,iPhone 13 Pro,Polycarbonate,<NA>
4,B09WJ1NBHQ,B096TZRWMZ,DE,265,2599.00,CASEKOO,グリーン・アルパイン,<NA>,iPhone 13 Pro,ポリカーボネート,<NA>
...,...,...,...,...,...,...,...,...,...,...,...
7175,B09C5RF4VN,B096V2N136,DE,867,16.99,TAURI,trasparente,<NA>,for iphone 13 6.1 inch,Poliuretano termoplastico,<NA>
7176,B09C5QXMQY,B096V2N136,DE,867,16.99,TAURI,D-Blue,<NA>,iPhone 13,Tempered Glass,<NA>
7177,B09C5QXMQY,B096V2N136,DE,867,17.99,TAURI,Azul,<NA>,iPhone 13,Vidrio templado,<NA>
7178,B09C5QXMQY,B096V2N136,DE,867,16.99,TAURI,Blu,<NA>,iPhone 13,Poliuretano termoplastico,<NA>


In [13]:
# combine the preprocessing steps into a single operator
processor = nvt.Workflow(categorical + numerical)
# fit the processor on the train variable
processor.fit(train)

AttributeError: DataFrame object has no attribute schema

In [26]:
total = categorical + numerical + 'id'
groupby_features =  (total
                     >> Groupby( 
                        groupby_cols=['id'],
                        aggs={
                            'prev_items': ['list','count','last'],
                            'next_item': ['list','count','last'],
                            'price': ['list'],
                            'brand': ['list'],
                            'color': ['list'],
                            'material': ['list'],
                            'author': ['list'],
                            'price': ['list'],
                            'size': ['list'],
                        },
                        sort_cols=['index']
                    )
                    # drop id column
                    >> Drop(columns=['id'])
                    # drop index column
                    >> Drop(columns=['index'])
)


TypeError: can only concatenate list (not "str") to list

In [19]:
ids

<Node Categorify output>